<a href="https://colab.research.google.com/github/fblondiaux/LGBIO2060-2020/blob/TP4-Draft/LGBIO2060_TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LGBIO2060 - Exercise session 4
Kalman filtering or how to implement the prior and likelihood dynamically? 


## Import and helper functions
**Please execute the cell below to initialize the notebook environment**

In [ ]:
#@title Imports
import numpy as np
import matplotlib.pyplot as plt

## Tutorial objectives 

At the end of this tutorial, you should be able to understand the 

# Section 1 - Model Formulation

## Section 2 - Cost Function

## Section 3 - Optimal Control

Now that our system is defined and that we have our cost function, we have to find the control gains, L that minimizes this cost function. 

They 


##Section 4 - State Estimation




Once again, our sensory inputs are not good enough to infer the position of the fly we are tracking. In order to have a better estimation of the fly's position at every time, we will use a continuous adaptation of Bayes' theorem: the **Kalman filter**. 

At every time step, the Kalman filter computes the weighted sum of the prior and the likelihood, weighted by a factor that characterises how trustable each term is. The result of this weighted sum will be the posterior distribution; this posterior distribution is the optimal estimator of the latent state of the system. 

The estimated state, denoted $\hat{x}$ is computed this way : 

$\begin{eqnarray}
& & \\
\hat{x}[t] & = & \text{trust}_{\text{prior}}x_{\text{prior}} + \text{trust}_{\text{likelihood}}x_{\text{likelihood}}\\
& & \\
\hat{x}[t+1] & = & A  \hat{x}[t]+ K[t]\left(y[t] - H \hat{x}[t]\right)\\
& & \\
\end{eqnarray}$

Where $K[t]$ is the Kalman gain evaluated at time $t$. These Kalman gains will be the weighting parameters that characterises how much we trust one source of information (prior and likelihood) over the other. Since these gains quantifies the trust we have in each source, they will be computed based on the covariances matrices of these two sources. The covariance matrix related to the prior is $\Omega_{\text{motor}}$ and the one related to the likelihood is $\Omega_{\text{sensory}}$. Therefore the gain of the kalman filter are recursively computed as follows:  

$\begin{eqnarray}
& & \\
K[t]& = &A\, \Sigma[t] \,H^T \left(H \,\Sigma[T]\,H^T+\Omega_{\text{motor}}\right)^{-1} \\
& & \\
\Sigma[t+1] & = & \Omega_{\text{sensory}} + \left(A-K[t]\,H\right)\Sigma[T]\,A^T
\end{eqnarray}$

Where $\Sigma[t] = \mathbb{C}\text{ov}\left\{\hat{x}[t]\right\}$ is the covariance matrix of the estimated state at time t. 


**Exercice 3 - Implement the function below to apply Kalman filter to the dynamical linear system**
